# RSA

Using this notebook:
- Code blocks have dark backgrounds.
- You can edit code blocks by clicking inside the block.
- You can execute a code block by clicking the "run" button (see above), or by pressing Shift+Enter while editing the block.
- The order in which code blocks are executed matters.  Normally we run from top to bottom.
- The order in which code blocks have been run follows nubmers to the left. For example, the code block with `In [1]` was the first one to be run. 

Some tips:
- To start over and execute all blocks, select Kernel -> Restart & Run All.

SAVING YOUR WORK:
- To save your work: click on the cloud icon with the downward pointing arrow.
- To restore your work: click on the cloud icon with the upward pointing arrow.
- If the system says "connection lost": save your work, open the URL again, and restore your work.

## Let's get started

We begin by importing the library `sympy`, a Python library for symbolic mathematics.

In [26]:
import sympy

### Public Key Generation

The first step in RSA requires that we choose two prime numbers, $p$ and $q$. 

To do this, typically we would start by generating a prime candidate with the number of desired bits. One way to do this is to randomly generate each bit, and then set the first and last bits to 1. This ensures that the generated number has both:
- the number of desired bits (don't want a 0 bit in the position with the highest positional value), and 
- is an odd number (even numbers are certainly not prime, except for 2 which is special).  

Then we need to test if the generated prime candidate is indeed a prime number. This is not straightforward. Typically a first test would check if the prime candidate is divisible by any of the known smaller primes. If it is, then the generated candidate is rejected and we start again.

If the prime candidate passes the first test, then a second test is performed. For very large prime candidates, it is impractical to check for divisibility of any prime number less than the candidate (and in fact for extremely large numbers we may not even know all of the prime numbers less than that number). This second test is not certain; it tells us only if the generated candidate is *likely* a prime number. To proceed with the assumption that the generated candidate is prime requires that the test return a high probability that the candidate is indeed prime. (In practice, the tolerance for uncertainty varies depending on the level of security required.)

For toy examples, we can instead use theory that tells us how to construct sets of prime numbers. Here we generate two Mersenne primes: prime numbers that have the form $2^n-1$ for some integer $n$. There is a finite list of integer exponents $n$ that will result in Mersenne primes, including $2, 3, 5, 7, 13, 17, 19, 31, \ldots$. To date there are only 51 Mersenne primes known, but it has been conjectured that there are in fact infinitely many. (Cool huh?)

In [27]:
p = (2**31) - 1
q = (2**61) - 1
print(f'p: {p}')
print(f'q: {q}')

p: 2147483647
q: 2305843009213693951


For these smaller prime numbers, we can use the function `isprime` to make sure that $p$ and $q$ are both prime:

In [28]:
print(f'p is prime: {sympy.isprime(p)}')
print(f'q is prime: {sympy.isprime(q)}')

p is prime: True
q is prime: True


Next we multiply $p$ and $q$ these to get $n$:

In [29]:
n = p * q
print(f'n: {n}')

n: 4951760154835678088235319297


Then we compute the totient of $n$, $\varphi(n)$. Recall that for a prime number $p$, $\varphi(p)=p-1$ because prime numbers are coprime with all of the $p-1$ positive integers less than themselves. Additionally, $\varphi(p\cdot q)=(p-1)\cdot (q-1)$. This means that since we know that $n= p \cdot q$ where $p$ and $q$ are prime (by construction), we can easily evaluate the totient of $n$.

In [30]:
phi = (q-1) * (p-1) # can also use: sympy.totient(n)
print(f'phi: {phi}')

phi: 4951760152529835076874141700


Next we need to choose an encryption exponent $e$ where: $e \in \mathbb{Z}$, $1 < e < n$, and $gcd(e, \varphi(n)) = 1$. In other words, choose an integer $e$ larger than 1, smaller than $n$, and is coprime with $\varphi(n)$. 

Coprimality of $e$ and $\varphi(n)$ is necessary to ensure the existence of a multiplicative inverse for $e$ in modulo $\varphi(n)$.

A straighforward way to ensure coprimality of $e$ and $\varphi(n)$ is to choose $e$ to be a prime number such that $1 < e < n$.

In [31]:
e = 17
gcd = sympy.gcd(e, phi)
print(f'e: {e}; gcd(e, phi): {gcd}')

e: 17; gcd(e, phi): 1


Now have our public key pair $\{e, n\}$. Share this with anyone, but whatever you do, do not share $p$ and $q$.

In [32]:
print(f'Public key pair: {{{e}, {n}}}')

Public key pair: {17, 4951760154835678088235319297}


### Private Key Generation

Computing the private key requires finding the inverse of $e$ mod $\varphi(n)$. We know this inverses exists, because $e$ was chosen to be coprime with $\varphi(n)$. In other words, we need to find $d$ such that

$e \cdot d \equiv 1 \pmod{\varphi(n)}$

where $e$ and $d$ are both residue classes modulo $\varphi(n)$ (so $e, d \in \{ [0], [1], \ldots, [ \varphi(n) - 1]\}$.

If we needed to find this multiplicative inverse by hand we would use the Backwards Euclidean Algorithm. However, here we get the machine to do it for us using the `mod_inverse` function:

In [33]:
d = sympy.mod_inverse(e, phi)
print(f'd: {d}')
print(f'd*e modulo phi: {(d*e)%phi}')

d: 4077920125612805357425763753
d*e modulo phi: 1


Now have our private key pair $\{d, n\}$.

In [34]:
print(f'Private key pair: {{{d}, {n}}}')

Private key pair: {4077920125612805357425763753, 4951760154835678088235319297}


### Encrypt A Message

Let's use our public key pair to encrypt a message. Here we will encrypt the message: "I like math". First, we map the message characters to ASCII decimal codes. Then we take that vector of ASCII codes and concatenate them to create a large integer version of the message:

In [35]:
message = "I LIKE MATH"
ordinal_message = [ ord(c) for c in message ]
large_integer_message = sum([ 100**i*o for i, o in enumerate(reversed(ordinal_message)) ])
print(f'message: {message}')
print(f'ascii ordinal version of message: {ordinal_message}')
print(f'large integer version of message: {large_integer_message}')

message: I LIKE MATH
ascii ordinal version of message: [73, 32, 76, 73, 75, 69, 32, 77, 65, 84, 72]
large integer version of message: 7332767375693277658472


Now we are ready to encrypt the message. To do this we use the power mod function which take three arguments, the base ($x$), the exponent ($e$), and the modulus ($n$) and returns $m^e \pmod{n}$.

In [36]:
encrypted_message = pow(large_integer_message, e, n)
print(f'encrypted message: {encrypted_message}')

encrypted message: 2079388683330327645114069204


Now we have the encrypted message.

### Decrypt The Message

To recover the original plaintext message, we use the power mod function again with the three arguments that are the base ($x$), the exponent ($d$), and the modulus ($n$). This time the base is the encrypted message. 

In [37]:
decrypted_message = pow(encrypted_message, d, n)
print(f'decrypted message: {decrypted_message}')
print(f'decrypted integer result equal to large integer message?: {decrypted_message==large_integer_message}')

decrypted message: 7332767375693277658472
decrypted integer result equal to large integer message?: True


Finally, let's convert the decrypted large integer back into plaintext by splitting the large integer into two-digit numbers, and then look up the corresponding characters for these decimal codes in the ASCII table.

In [38]:
a = decrypted_message
ords = []
while a:
    ords.append(a % 100)
    a //= 100
recovered_message = ''.join(reversed([ chr(i) for i in ords ]))
print(f'recovered message: {recovered_message}')


recovered message: I LIKE MATH
